## Classify Quickdraw clouds vs pencils

In [160]:
# https://towardsdatascience.com/neural-networks-with-google-colaboratory-artificial-intelligence-getting-started-713b5eb07f14
# !pip install -U -q PyDrive
# !pip install -q keras
# !pip install -q h5py
!pip install -q libhdf5
!pip show h5py

  Could not find a version that satisfies the requirement libhdf5 (from versions: )
No matching distribution found for libhdf5
Name: h5py
Version: 2.7.1
Summary: Read and write HDF5 files from Python
Home-page: http://www.h5py.org
Author: Andrew Collette
Author-email: andrew.collette@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, six


In [141]:
!pip show tensorflow
import math

import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Name: tensorflow
Version: 1.4.1
Summary: TensorFlow helps the tensors flow
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: enum34, numpy, tensorflow-tensorboard, six, protobuf, wheel


In [0]:
# cloud = np.load('%s.npy' % ('cloud',)) / 255
# pencil = np.load('%s.npy' % ('pencil',)) / 255


cloud_file = drive.CreateFile({'id': '1GMP27LtfkbDR4z7BwZkiR3m_3C0sqFIM'})
cloud_file.GetContentFile('cloud.npy')
pencil_file = drive.CreateFile({'id': '1f0wxJyU13GoVDDxtyR5pTgmQxav6-Nu6'})
pencil_file.GetContentFile('pencil.npy')

cloud = np.load('%s.npy' % ('cloud',)) / 255
pencil = np.load('%s.npy' % ('pencil',)) / 255

N = {'cloud': cloud.shape[0], 'pencil': pencil.shape[0]}
N_total = sum([n_item for item, n_item in N.items()])
N_pixels = cloud.shape[1]

k = int(math.sqrt(N_pixels))
cloud = cloud.reshape((N['cloud'], k, k, 1))
pencil = pencil.reshape((N['pencil'], k, k, 1))

In [0]:
# print(cloud.shape, pencil.shape)
# print(cloud[:1], '\n', pencil[:1])

In [144]:
# X = np.stack([cloud, pencil], axis=0) # Don't use stack - this joins along 3rd axis!
X = np.concatenate([cloud, pencil], axis=0)
print(X.shape)

y = np.zeros([N_total, 1])
y[0:N['cloud']] = 0
# y[N['cloud']:N['pencil']] = 1 # This is wrong!
y[N['cloud']:N['cloud'] + N['pencil']] = 1

enc = OneHotEncoder()
# I don't understand the display without .toarray()
# https://stackoverflow.com/questions/45738308/scikit-convert-one-hot-encoding-to-encoding-with-integers
y = enc.fit_transform(y).toarray()
print(y[:10], '\n', y[N['cloud']:N['cloud']+10])

(242266, 28, 28, 1)
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]] 
 [[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [145]:
n0y = 0
n1y = 0
for sample in y:
  if sample[0] > 0: n0y += 1
  else: n1y += 1
print(n0y, n1y)

120265 122001


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=100)
# print(y_train.shape, y_test.shape)
# print(X_train[:12], '\n', y_train[:12])
# print(y_train[N['cloud']:N['cloud']+10])
# print(y_train[N['cloud']+1000:N['cloud']+1010])
# print(y_train[:10], '\n', y_train[N['cloud']:N['cloud']+10])
# print(y_train[-10:])

# X_train = np.expand_dims(X_train, axis=3)
# X_test = np.expand_dims(X_test, axis=3)
# y_train = np.expand_dims(y_train, axis=3)
# y_test = np.expand_dims(y_test, axis=3)

In [154]:
n0 = 0
n1 = 0
for sample in y_train:
  if sample[0] > 0: n0 += 1
  else: n1 += 1
print(n0, n1)

n0 = 0
n1 = 0
for sample in y_test:
  if sample[0] > 0: n0 += 1
  else: n1 += 1
print(n0, n1)

119078 120765
1187 1236


In [0]:
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, \
                         ZeroPadding2D, BatchNormalization, Flatten, \
                         Dropout

In [0]:
def quickdraw_model(shape, drop=0.5): # How does it know # of classes?
  X_input = Input(shape)
  
  X = ZeroPadding2D((1,1))(X_input)
  
  X = Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), name = 'conv0')(X)
  X = BatchNormalization(axis = 3, name = 'bn0')(X)
  X = Dropout(drop)(X)
  X = Activation('relu')(X)
  
  X = MaxPooling2D((2,2), name='max_pool0')(X) # X is 14 x 14
  
  X = ZeroPadding2D((3,3))(X)
  
  X = Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), name = 'conv1')(X)
  X = BatchNormalization(axis = 3, name = 'bn1')(X)
  X = Dropout(drop)(X)
  X = Activation('relu')(X)
  
  X = MaxPooling2D((2,2), name='max_pool1')(X) # X is 7 x 7
  
  X = Flatten()(X)
  X = Dense(units=2, activation='sigmoid', name='fc0')(X)
  
  model = Model(inputs=X_input, outputs=X, name='QuickdrawModel')
  
  return model

In [0]:
model = quickdraw_model((k, k, 1))
model.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [151]:
model.fit(x=X_train, y=y_train, epochs=1, batch_size=500)

Epoch 1/1
239843/239843 [==============================] - 692s 3ms/step - loss: 0.0707 - acc: 0.9743


In [0]:
# Model without dropout, 8 and 16 conv filters respectively
# model.fit(x=X_train, y=y_train, epochs=1, batch_size=500) 
# Epoch 1/1
# 239843/239843 [==============================] - 328s 1ms/step - loss: 0.0732 - acc: 0.9721
# <keras.callbacks.History at 0x7fec7fee1a90>
# 2423/2423 [==============================] - 1s 550us/step
# Loss = 0.459567809014
# Test Accuracy = 0.811390837534

# Model with dropout, 16 and 32 conv filters respectively
# Epoch 1/1
# 239843/239843 [==============================] - 692s 3ms/step - loss: 0.0707 - acc: 0.9743
# <keras.callbacks.History at 0x7fec7c7815c0>
# 2423/2423 [==============================] - 3s 1ms/step
# Loss = 0.342920201527
# Test Accuracy = 0.841312422813



In [152]:
preds = model.evaluate(x=X_test, y=y_test)
### END CODE HERE ###
print()
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))
# Why is the test accuracy so low even though the train acc is 97% on one epoch?

2423/2423 [==============================] - 3s 1ms/step

Loss = 0.342920201527
Test Accuracy = 0.841312422813


In [158]:
model.save('quickdraw_model.h5')

ImportError: ignored